In [6]:
"""
🔴 AUTO-CAPTURE V4 - OPTIMIZED for RED Dragon Fruit Detection
FIXED: Deteksi buah naga merah ASLI (glossy, dark, berbagai pencahayaan)

Key Improvements:
1. ✅ Expanded RED color range (cover dark & bright red)
2. ✅ Brightness normalization (consistent detection)
3. ✅ Very lenient object detection (for glossy surface)
4. ✅ Lower area threshold (12% instead of 20%)
5. ✅ Reject screen/display based on texture
"""

import cv2
import numpy as np
import joblib
from scipy.stats import skew
from collections import deque
import time
import os
from datetime import datetime

# ---------- 1. LOAD MODEL ----------
MODEL_PATH = "knn_buah_naga_optimized.pkl"

try:
    model = joblib.load(MODEL_PATH)
    print(f"✅ Model loaded: {MODEL_PATH}")
    
    if hasattr(model, 'classes_'):
        classes = model.classes_
    elif hasattr(model, 'named_steps'):
        knn = model.named_steps.get('knn')
        classes = knn.classes_ if knn and hasattr(knn, 'classes_') else []
    else:
        classes = []
    
    print(f"   Classes: {classes}")
except Exception as e:
    print(f"❌ Error: {e}")
    exit(1)

# ---------- 2. CONFIGURATION (OPTIMIZED FOR REAL FRUIT) ----------
TARGET_SIZE = (800, 800)
ROI_SIZE = 350
CONFIDENCE_THRESHOLD = 65.0         # Lower threshold untuk real fruit
CAPTURE_COOLDOWN = 3.0
MOTION_THRESHOLD = 5000
FRAME_BUFFER_SIZE = 7

# 🔴 OPTIMIZED FOR RED DRAGON FRUIT
MIN_DRAGON_FRUIT_AREA = 12.0        # 12% (turun dari 20%) - akomodasi refleksi
MIN_VARIANCE = 150                   # Very lenient - glossy surface OK
MIN_EDGE_DENSITY = 0.015             # Very lenient - smooth surface OK
MAX_VARIANCE = 2000                  # NEW! Reject screen/display (texture terlalu tinggi)

BASE_CAPTURE_DIR = "auto_captures"
os.makedirs(BASE_CAPTURE_DIR, exist_ok=True)

prediction_buffer = deque(maxlen=FRAME_BUFFER_SIZE)
capture_history = deque(maxlen=5)
last_capture_time = 0
prev_frame_gray = None

# ---------- 3. HELPER FUNCTIONS ----------
def bgr_to_hsi(bgr_image):
    """BGR to HSI conversion"""
    b = bgr_image[:, :, 0].astype(np.float32) / 255.0
    g = bgr_image[:, :, 1].astype(np.float32) / 255.0
    r = bgr_image[:, :, 2].astype(np.float32) / 255.0

    i = (r + g + b) / 3.0
    min_rgb = np.minimum(np.minimum(r, g), b)
    denominator = r + g + b + 1e-6
    s = 1 - (3.0 * min_rgb) / denominator
    s = np.clip(s, 0, 1)

    numerator = np.sqrt((r - g) ** 2 + (r - b) * (g - b))
    valid = numerator > 1e-6
    h = np.zeros_like(numerator)
    cos_theta = np.where(valid, ((r - g) + (r - b)) / (2.0 * numerator + 1e-6), 1.0)
    cos_theta = np.clip(cos_theta, -1.0, 1.0)
    h = np.degrees(np.arccos(cos_theta))
    h[b > g] = 360 - h[b > g]
    h = np.clip(h, 0, 360)
    return h, s, i

def extract_color_features(bgr_img):
    """Extract 18 color features"""
    img = cv2.resize(bgr_img, TARGET_SIZE, interpolation=cv2.INTER_AREA)
    
    features = []
    def get_stats(channel):
        mean_val = np.mean(channel)
        std_val = np.std(channel)
        skew_val = skew(channel.flatten()) if std_val > 1e-6 else 0.0
        return mean_val, std_val, skew_val
    
    b, g, r = cv2.split(img)
    for channel in [r, g, b]:
        features.extend(get_stats(channel))
    
    h, s, i_channel = bgr_to_hsi(img)
    for channel in [h, s, i_channel]:
        features.extend(get_stats(channel))
    
    return np.array(features)

# ---------- 4. MOTION DETECTION ----------
def detect_motion(current_frame, prev_frame, roi_coords):
    """Motion detection"""
    if prev_frame is None:
        return True, 0
    
    x1, y1, x2, y2 = roi_coords
    gray1 = cv2.cvtColor(current_frame[y1:y2, x1:x2], cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(prev_frame[y1:y2, x1:x2], cv2.COLOR_BGR2GRAY)
    
    diff = cv2.absdiff(gray1, gray2)
    _, thresh = cv2.threshold(diff, 30, 255, cv2.THRESH_BINARY)
    changed_pixels = np.sum(thresh > 0)
    
    return changed_pixels > MOTION_THRESHOLD, changed_pixels

# ---------- 5. PREDICTION ----------
def predict_maturity(roi):
    """Predict with smoothing"""
    try:
        features = extract_color_features(roi)
        label = model.predict([features])[0]
        
        if hasattr(model, 'predict_proba'):
            proba = model.predict_proba([features])[0]
            confidence = np.max(proba) * 100
            
            if hasattr(model, 'classes_'):
                classes = model.classes_
            elif hasattr(model, 'named_steps'):
                knn = model.named_steps.get('knn')
                classes = knn.classes_ if knn else []
            else:
                classes = []
            
            class_probs = {cls: prob * 100 for cls, prob in zip(classes, proba)}
        else:
            confidence = 100.0
            class_probs = {label: 100.0}
        
        prediction_buffer.append((label, confidence))
        
        labels = [pred[0] for pred in prediction_buffer]
        from collections import Counter
        label_counts = Counter(labels)
        smoothed_label = label_counts.most_common(1)[0][0]
        
        confidences = [pred[1] for pred in prediction_buffer if pred[0] == smoothed_label]
        smoothed_confidence = np.mean(confidences) if confidences else confidence
        
        return smoothed_label, smoothed_confidence, features, class_probs
    except Exception as e:
        print(f"[ERROR]: {e}")
        return "ERROR", 0.0, None, {}

# ---------- 6. 🔴 ENHANCED RED DRAGON FRUIT COLOR DETECTION ----------
def is_dragon_fruit_color(roi):
    """
    🔴 OPTIMIZED untuk buah naga merah ASLI
    
    Covers:
    - Dark red (mature): H=0-10, 170-180, S=20-100, V=30-90
    - Bright red (ripe): H=0-15, 165-180, S=30-100, V=70-255
    - Pink (immature): H=340-360, S=20-80, V=60-200
    
    Returns: (is_dragon_fruit, color_type, percentage, debug_info)
    """
    # Brightness normalization untuk consistent detection
    lab = cv2.cvtColor(roi, cv2.COLOR_BGR2LAB)
    l, a, b_ch = cv2.split(lab)
    l = cv2.equalizeHist(l)  # Normalize luminance
    lab_normalized = cv2.merge([l, a, b_ch])
    roi_normalized = cv2.cvtColor(lab_normalized, cv2.COLOR_LAB2BGR)
    
    # Convert to HSV
    hsv = cv2.cvtColor(roi_normalized, cv2.COLOR_BGR2HSV)
    
    # 🔴 RED COLOR RANGES (EXPANDED!)
    
    # Dark Red (mature dragon fruit) - merah gelap
    # H: 0-10 & 170-180 (red wrap), S: 20-100 (akomodasi glossy), V: 30-90 (dark)
    mask_dark_red1 = cv2.inRange(hsv, np.array([0, 20, 30]), np.array([10, 100, 90]))
    mask_dark_red2 = cv2.inRange(hsv, np.array([170, 20, 30]), np.array([180, 100, 90]))
    mask_dark_red = cv2.bitwise_or(mask_dark_red1, mask_dark_red2)
    
    # Bright Red (ripe dragon fruit) - merah cerah
    # H: 0-15 & 165-180, S: 30-100, V: 70-255 (bright)
    mask_bright_red1 = cv2.inRange(hsv, np.array([0, 30, 70]), np.array([15, 100, 255]))
    mask_bright_red2 = cv2.inRange(hsv, np.array([165, 30, 70]), np.array([180, 100, 255]))
    mask_bright_red = cv2.bitwise_or(mask_bright_red1, mask_bright_red2)
    
    # Pink/Light Red (less mature) - merah muda
    # H: 0-5 & 175-180, S: 15-60, V: 80-255
    mask_pink1 = cv2.inRange(hsv, np.array([0, 15, 80]), np.array([5, 60, 255]))
    mask_pink2 = cv2.inRange(hsv, np.array([175, 15, 80]), np.array([180, 60, 255]))
    mask_pink = cv2.bitwise_or(mask_pink1, mask_pink2)
    
    # Calculate percentages
    total_pixels = roi.shape[0] * roi.shape[1]
    dark_red_percent = (np.sum(mask_dark_red > 0) / total_pixels) * 100
    bright_red_percent = (np.sum(mask_bright_red > 0) / total_pixels) * 100
    pink_percent = (np.sum(mask_pink > 0) / total_pixels) * 100
    
    # Combine all red masks
    mask_all_red = cv2.bitwise_or(mask_dark_red, mask_bright_red)
    mask_all_red = cv2.bitwise_or(mask_all_red, mask_pink)
    total_red_percent = (np.sum(mask_all_red > 0) / total_pixels) * 100
    
    # Also check for yellow/green (untuk buah naga kuning/putih)
    mask_yellow = cv2.inRange(hsv, np.array([20, 30, 60]), np.array([40, 255, 255]))
    mask_green = cv2.inRange(hsv, np.array([40, 20, 40]), np.array([80, 180, 220]))
    
    yellow_percent = (np.sum(mask_yellow > 0) / total_pixels) * 100
    green_percent = (np.sum(mask_green > 0) / total_pixels) * 100
    
    # Total dragon fruit color (red + yellow + green)
    mask_dragon_fruit = cv2.bitwise_or(mask_all_red, mask_yellow)
    mask_dragon_fruit = cv2.bitwise_or(mask_dragon_fruit, mask_green)
    dragon_fruit_percent = (np.sum(mask_dragon_fruit > 0) / total_pixels) * 100
    
    # Determine color type
    color_percentages = {
        'Dark Red': dark_red_percent,
        'Bright Red': bright_red_percent,
        'Pink': pink_percent,
        'Yellow': yellow_percent,
        'Green': green_percent
    }
    dominant_color = max(color_percentages, key=color_percentages.get)
    
    # 🎯 VALIDATION: Minimal 12% (turun dari 20%)
    is_dragon_fruit = dragon_fruit_percent >= MIN_DRAGON_FRUIT_AREA
    
    debug_info = {
        'dark_red': dark_red_percent,
        'bright_red': bright_red_percent,
        'pink': pink_percent,
        'yellow': yellow_percent,
        'green': green_percent,
        'total_red': total_red_percent,
        'total': dragon_fruit_percent
    }
    
    return is_dragon_fruit, dominant_color, dragon_fruit_percent, debug_info

# ---------- 7. OBJECT DETECTION (VERY LENIENT + REJECT SCREEN) ----------
def is_object_present(roi):
    """
    Object detection dengan:
    - Very lenient threshold (buah asli OK)
    - Reject screen/display (variance terlalu tinggi)
    """
    gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    variance = np.var(gray)
    
    edges = cv2.Canny(gray, 50, 150)
    edge_density = np.sum(edges > 0) / edges.size
    
    # Check variance range
    has_variance_ok = MIN_VARIANCE < variance < MAX_VARIANCE
    has_edges_ok = edge_density > MIN_EDGE_DENSITY
    
    # Reject if variance TOO HIGH (likely screen/display)
    is_screen = variance > MAX_VARIANCE
    
    is_valid_object = has_variance_ok and has_edges_ok and not is_screen
    
    return is_valid_object, variance, edge_density, is_screen

# ---------- 8. AUTO-CAPTURE LOGIC ----------
def auto_capture(roi, label, confidence, class_probs, color_valid, color_percent, is_screen):
    """
    Auto capture with validation:
    1. Not a screen/display
    2. Dragon fruit color detected
    3. High confidence
    """
    global last_capture_time
    
    current_time = time.time()
    time_since_last = current_time - last_capture_time
    
    # Reject screen/display
    if is_screen:
        return False, "❌ Screen/Display detected (texture too high)"
    
    # Color validation
    if not color_valid:
        return False, f"❌ Color: {color_percent:.1f}% (need ≥{MIN_DRAGON_FRUIT_AREA}%)"
    
    # Confidence check
    if confidence < CONFIDENCE_THRESHOLD:
        return False, f"Low confidence: {confidence:.1f}%"
    
    # Cooldown
    if time_since_last < CAPTURE_COOLDOWN:
        return False, f"Cooldown: {CAPTURE_COOLDOWN - time_since_last:.1f}s"
    
    # CAPTURE!
    class_dir = os.path.join(BASE_CAPTURE_DIR, label)
    os.makedirs(class_dir, exist_ok=True)
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"{label}_{confidence:.0f}_{timestamp}.jpg"
    filepath = os.path.join(class_dir, filename)
    
    cv2.imwrite(filepath, roi)
    last_capture_time = current_time
    
    capture_info = {
        'time': datetime.now().strftime("%H:%M:%S"),
        'label': label,
        'confidence': confidence,
        'filename': filename
    }
    capture_history.append(capture_info)
    
    print(f"\n[📸 CAPTURED] {filepath}")
    print(f"   {label} | {confidence:.1f}% | Color: {color_percent:.1f}%")
    
    return True, "✅ Captured!"

# ---------- 9. VISUAL HELPERS ----------
def get_color_by_status(color_valid, confidence, is_screen):
    """ROI box color"""
    if is_screen:
        return (0, 0, 255)        # Red - screen detected
    elif not color_valid:
        return (100, 100, 100)    # Gray - not dragon fruit
    elif confidence >= CONFIDENCE_THRESHOLD:
        return (0, 255, 0)        # Green - ready
    else:
        return (0, 255, 255)      # Yellow - low confidence

def draw_status_panel(frame, label, confidence, class_probs, color_valid, color_info, obj_info):
    """Status panel with debug info"""
    h, w = frame.shape[:2]
    
    overlay = frame.copy()
    cv2.rectangle(overlay, (10, 10), (500, 420), (0, 0, 0), -1)
    frame = cv2.addWeighted(overlay, 0.75, frame, 0.25, 0)
    
    y = 30
    
    # Title
    cv2.putText(frame, "AUTO-CAPTURE V4 - Red Dragon Fruit", 
                (20, y), cv2.FONT_HERSHEY_SIMPLEX, 0.55, (100, 200, 255), 2)
    y += 35
    
    # Layer 1: Object Detection
    cv2.putText(frame, "Layer 1: Object Detection", 
                (20, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (200, 200, 200), 1)
    y += 20
    obj_present, variance, edge_density, is_screen = obj_info
    
    if is_screen:
        status = "REJECT - SCREEN"
        color = (0, 0, 255)
    elif obj_present:
        status = "PASS"
        color = (0, 255, 0)
    else:
        status = "FAIL"
        color = (0, 0, 255)
    
    cv2.putText(frame, f"  Status: {status}", 
                (20, y), cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 1)
    y += 18
    cv2.putText(frame, f"  Var:{variance:.0f} (range:{MIN_VARIANCE}-{MAX_VARIANCE}) Edge:{edge_density:.3f}", 
                (20, y), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (180, 180, 180), 1)
    y += 25
    
    # Layer 2: Color Validation
    cv2.putText(frame, "Layer 2: Red Color Detection (Enhanced)", 
                (20, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (200, 200, 200), 1)
    y += 20
    color_status = "PASS" if color_valid else "FAIL"
    color_color = (0, 255, 0) if color_valid else (0, 0, 255)
    cv2.putText(frame, f"  Status: {color_status} | Total: {color_info['total']:.1f}%", 
                (20, y), cv2.FONT_HERSHEY_SIMPLEX, 0.45, color_color, 1)
    y += 18
    cv2.putText(frame, f"  Dark Red: {color_info['dark_red']:.1f}% | Bright Red: {color_info['bright_red']:.1f}%", 
                (20, y), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (180, 180, 180), 1)
    y += 16
    cv2.putText(frame, f"  Pink: {color_info['pink']:.1f}% | Total Red: {color_info['total_red']:.1f}%", 
                (20, y), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (180, 180, 180), 1)
    y += 16
    cv2.putText(frame, f"  Min Required: {MIN_DRAGON_FRUIT_AREA}%", 
                (20, y), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (150, 150, 150), 1)
    y += 25
    
    # Layer 3: KNN Prediction
    cv2.putText(frame, "Layer 3: KNN Prediction", 
                (20, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (200, 200, 200), 1)
    y += 20
    conf_status = "PASS" if confidence >= CONFIDENCE_THRESHOLD else "FAIL"
    conf_color = (0, 255, 0) if confidence >= CONFIDENCE_THRESHOLD else (0, 165, 255)
    cv2.putText(frame, f"  Status: {conf_status} | Label: {label} ({confidence:.1f}%)", 
                (20, y), cv2.FONT_HERSHEY_SIMPLEX, 0.45, conf_color, 1)
    y += 22
    
    # Probabilities
    for cls, prob in sorted(class_probs.items(), key=lambda x: x[1], reverse=True):
        bar_width = int(prob * 3.5)
        cv2.rectangle(frame, (20, y - 10), (20 + bar_width, y + 4), 
                     (0, 255, 0) if prob > 50 else (0, 165, 255), -1)
        cv2.putText(frame, f"  {cls}: {prob:.1f}%", 
                   (160, y), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1)
        y += 18
    
    # Final result
    y += 10
    all_pass = obj_present and not is_screen and color_valid and confidence >= CONFIDENCE_THRESHOLD
    if is_screen:
        result = "RESULT: Screen/Display Rejected"
        result_color = (0, 0, 255)
    elif all_pass:
        result = "RESULT: READY TO CAPTURE!"
        result_color = (0, 255, 0)
    else:
        result = "RESULT: Validation Failed"
        result_color = (0, 100, 255)
    
    cv2.putText(frame, result, 
                (20, y), cv2.FONT_HERSHEY_SIMPLEX, 0.55, result_color, 2)
    
    return frame

def draw_capture_history(frame):
    """History panel"""
    if len(capture_history) == 0:
        return frame
    
    h, w = frame.shape[:2]
    overlay = frame.copy()
    panel_height = min(30 + len(capture_history) * 22, 180)
    cv2.rectangle(overlay, (w - 450, h - panel_height - 10), 
                 (w - 10, h - 10), (0, 0, 0), -1)
    frame = cv2.addWeighted(overlay, 0.75, frame, 0.25, 0)
    
    y = h - panel_height
    cv2.putText(frame, "CAPTURE HISTORY:", 
                (w - 440, y), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (100, 200, 255), 1)
    y += 20
    
    for cap in reversed(list(capture_history)):
        text = f"{cap['time']} | {cap['label']} ({cap['confidence']:.0f}%)"
        cv2.putText(frame, text, 
                   (w - 440, y), cv2.FONT_HERSHEY_SIMPLEX, 0.38, (200, 200, 200), 1)
        y += 18
    
    return frame

# ---------- 10. MAIN LOOP ----------
def main():
    global prev_frame_gray, last_capture_time, CONFIDENCE_THRESHOLD
    
    cap = cv2.VideoCapture(0)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
    
    if not cap.isOpened():
        print("❌ Camera failed")
        return
    
    print("\n" + "="*70)
    print("🔴 AUTO-CAPTURE V4 - Red Dragon Fruit Detection")
    print("="*70)
    print(f"Model: {MODEL_PATH}")
    print(f"Confidence: {CONFIDENCE_THRESHOLD}%")
    print(f"Color area: ≥{MIN_DRAGON_FRUIT_AREA}%")
    print(f"Variance range: {MIN_VARIANCE}-{MAX_VARIANCE} (reject screen if > {MAX_VARIANCE})")
    print("\nOptimized for:")
    print("  ✅ Dark red (mature)")
    print("  ✅ Bright red (ripe)")
    print("  ✅ Pink (less mature)")
    print("  ✅ Glossy surface")
    print("  ❌ Screen/Display (auto reject)")
    print("\nControls: q=quit, m=manual, r=reset, +/-=threshold")
    print("-" * 70)
    
    frame_count = 0
    fps_start = time.time()
    fps = 0
    
    label, confidence, class_probs = "WAITING", 0.0, {}
    color_valid, color_info = False, {}
    obj_info = (False, 0, 0, False)
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        frame_count += 1
        h, w = frame.shape[:2]
        
        if frame_count % 30 == 0:
            fps = 30 / (time.time() - fps_start)
            fps_start = time.time()
        
        # ROI
        size = ROI_SIZE
        x1, y1 = (w - size) // 2, (h - size) // 2
        x2, y2 = x1 + size, y1 + size
        x1, y1 = max(0, x1), max(0, y1)
        x2, y2 = min(w, x2), min(h, y2)
        
        roi = frame[y1:y2, x1:x2]
        if roi.size == 0:
            continue
        
        # Motion
        motion_detected, _ = detect_motion(frame, prev_frame_gray, (x1, y1, x2, y2))
        prev_frame_gray = frame.copy()
        
        # Layer 1: Object detection
        obj_present, variance, edge_density, is_screen = is_object_present(roi)
        obj_info = (obj_present, variance, edge_density, is_screen)
        
        # Layer 2: Color validation
        if obj_present and not is_screen:
            color_valid, dominant_color, color_percent, debug_info = is_dragon_fruit_color(roi)
            color_info = debug_info
            color_info['dominant'] = dominant_color
            
            # Layer 3: Prediction
            if color_valid and (motion_detected or frame_count % 5 == 0):
                label, confidence, _, class_probs = predict_maturity(roi)
                
                # Auto-capture
                captured, status = auto_capture(roi, label, confidence, class_probs, 
                                               color_valid, color_percent, is_screen)
                if captured:
                    flash = np.ones_like(frame) * 255
                    frame = cv2.addWeighted(frame, 0.6, flash, 0.4, 0)
        else:
            color_valid = False
            color_info = {'dark_red': 0, 'bright_red': 0, 'pink': 0, 'yellow': 0, 'green': 0, 'total_red': 0, 'total': 0}
            if is_screen:
                label = "SCREEN DETECTED"
            else:
                label = "NO OBJECT"
            confidence, class_probs = 0.0, {}
        
        # Draw ROI
        box_color = get_color_by_status(color_valid, confidence, is_screen)
        thickness = 4 if (color_valid and confidence >= CONFIDENCE_THRESHOLD and not is_screen) else 2
        cv2.rectangle(frame, (x1, y1), (x2, y2), box_color, thickness)
        
        center_x, center_y = (x1 + x2) // 2, (y1 + y2) // 2
        cv2.drawMarker(frame, (center_x, center_y), box_color, cv2.MARKER_CROSS, 30, 3)
        
        # Status text
        if is_screen:
            status_text = "SCREEN/DISPLAY DETECTED - REJECTED"
            status_color = (0, 0, 255)
        elif not obj_present:
            status_text = "WAITING FOR OBJECT"
            status_color = (100, 100, 100)
        elif not color_valid:
            status_text = f"NOT RED DRAGON FRUIT ({color_info['total']:.1f}%)"
            status_color = (0, 100, 255)
        elif confidence < CONFIDENCE_THRESHOLD:
            status_text = f"LOW CONFIDENCE ({confidence:.1f}%)"
            status_color = (0, 255, 255)
        else:
            status_text = "READY TO CAPTURE!"
            status_color = (0, 255, 0)
        
        cv2.putText(frame, status_text, (x1 - 100, y1 - 15), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.65, status_color, 2)
        
        # Panels
        frame = draw_status_panel(frame, label, confidence, class_probs, 
                                 color_valid, color_info, obj_info)
        frame = draw_capture_history(frame)
        
        # FPS
        cv2.putText(frame, f"FPS: {fps:.1f}", (w - 120, 30), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
        
        cv2.imshow("Red Dragon Fruit Scanner V4", frame)
        
        # Keyboard
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break
        elif key == ord('m'):
            if color_valid and confidence >= CONFIDENCE_THRESHOLD and not is_screen:
                last_capture_time = 0
                auto_capture(roi, label, confidence, class_probs, color_valid, color_info['total'], is_screen)
        elif key == ord('r'):
            capture_history.clear()
        elif key == ord('+') or key == ord('='):
            CONFIDENCE_THRESHOLD = min(95, CONFIDENCE_THRESHOLD + 5)
            print(f"[⚙️] Threshold: {CONFIDENCE_THRESHOLD}%")
        elif key == ord('-') or key == ord('_'):
            CONFIDENCE_THRESHOLD = max(50, CONFIDENCE_THRESHOLD - 5)
            print(f"[⚙️] Threshold: {CONFIDENCE_THRESHOLD}%")
    
    cap.release()
    cv2.destroyAllWindows()
    
    print("\n" + "="*70)
    print(f"Stopped | Frames: {frame_count} | Captures: {len(capture_history)}")
    if len(capture_history) > 0:
        for cap in capture_history:
            print(f"  {cap['time']} | {cap['label']} ({cap['confidence']:.1f}%)")
    print("="*70)

if __name__ == "__main__":
    main()

✅ Model loaded: knn_buah_naga_optimized.pkl
   Classes: ['belum_matang' 'matang_sempurna' 'setengah_matang']

🔴 AUTO-CAPTURE V4 - Red Dragon Fruit Detection
Model: knn_buah_naga_optimized.pkl
Confidence: 65.0%
Color area: ≥12.0%
Variance range: 150-2000 (reject screen if > 2000)

Optimized for:
  ✅ Dark red (mature)
  ✅ Bright red (ripe)
  ✅ Pink (less mature)
  ✅ Glossy surface
  ❌ Screen/Display (auto reject)

Controls: q=quit, m=manual, r=reset, +/-=threshold
----------------------------------------------------------------------

[📸 CAPTURED] auto_captures\matang_sempurna\matang_sempurna_100_20251105_212348.jpg
   matang_sempurna | 100.0% | Color: 42.7%

[📸 CAPTURED] auto_captures\matang_sempurna\matang_sempurna_100_20251105_212352.jpg
   matang_sempurna | 100.0% | Color: 42.0%

[📸 CAPTURED] auto_captures\matang_sempurna\matang_sempurna_96_20251105_212357.jpg
   matang_sempurna | 96.1% | Color: 49.1%

[📸 CAPTURED] auto_captures\matang_sempurna\matang_sempurna_72_20251105_212400.jpg
